# Function to automatic merge and fill missing values



## Create the function
You have to create a function which take 2 arguments :
- argument 1 : DataFrame main
- argument 2 : DataFrame opinion

Your function will return a new DataFrame which is the left merge of both DataFrames, and which has no missing values (so your function has to fill them), add new columns (flag, MONTH and DAY), and OPINION should be translated. 

So you have to copy each useful code from the previous quests, and integrate it into a function. **This should achieve the same results as the previous quests.**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

link_main = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather_main_2018.csv"
link_opinion = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather_opinion_2018.csv"
df_main = pd.read_csv(link_main)
df_opinion = pd.read_csv(link_opinion)
print(df_main.head())
print(df_opinion.head())

         DATE  MAX_TEMPERATURE_C  ...  DEWPOINT_MAX_C  WINDTEMP_MAX_C
0  2018-01-01                 12  ...               8               7
1  2018-01-02                 13  ...              12               6
2  2018-01-03                 15  ...              13               7
3  2018-01-04                 14  ...              12              10
4  2018-01-05                 12  ...              10               7

[5 rows x 15 columns]
         date  WEATHER_CODE_EVENING  TOTAL_SNOW_MM  UV_INDEX  SUNHOUR OPINION
0  2018-01-01                   113              0         3      5.1     bad
1  2018-03-12                   119              0         2      8.8     bad
2  2018-03-09                   116              0         3     10.2     bad
3  2018-10-07                   122              0         1      5.6     bad
4  2018-06-18                   119              0         1     12.9     bad


In [ ]:
def merge_and_fill(DataFrameMain, DataFrameOpinion):
  NewDataFrame = pd.merge(DataFrameMain, DataFrameOpinion, how='left', left_on='DATE', right_on='date')
  NewDataFrame['flag'] = NewDataFrame['OPINION'].isna()
  mean_weather_column = NewDataFrame['WEATHER_CODE_EVENING'].mean()
  NewDataFrame['WEATHER_CODE_EVENING'].fillna(mean_weather_column, inplace = True)
  mean_snow_column = NewDataFrame['TOTAL_SNOW_MM'].mean()
  NewDataFrame['TOTAL_SNOW_MM'].fillna(mean_snow_column, inplace = True)
  mean_uv_column = NewDataFrame['UV_INDEX'].mean()
  NewDataFrame['UV_INDEX'].fillna(mean_uv_column, inplace = True)
  NewDataFrame['SUNHOUR'].fillna(method="ffill", inplace = True)
  NewDataFrame['OPINION'].fillna('unknown', inplace = True)

  def translation(word):
    dico = {"very bad": "très mauvais", "bad": "mauvais", "not good not bad": "moyen", "good": "bon", "very good": "très bon"}
    for key, value in dico.items():
      if word == key:
        return value
    return word

  NewDataFrame['OPINION'] = NewDataFrame['OPINION'].apply(translation)

  def find_month(date):
    transformed_date = datetime.strptime(date, '%Y-%m-%d').month
    return transformed_date

  NewDataFrame['MONTH'] = NewDataFrame['DATE'].apply(find_month)

  def find_day(date):
    transformed_date = datetime.strptime(date, '%Y-%m-%d').day
    return transformed_date

  NewDataFrame['DAY'] = NewDataFrame['DATE'].apply(find_day)

  return NewDataFrame

merge_and_fill(df_main, df_opinion)

,DATE,MAX_TEMPERATURE_C,MIN_TEMPERATURE_C,WINDSPEED_MAX_KMH,TEMPERATURE_MORNING_C,TEMPERATURE_NOON_C,TEMPERATURE_EVENING_C,PRECIP_TOTAL_DAY_MM,HUMIDITY_MAX_PERCENT,VISIBILITY_AVG_KM,PRESSURE_MAX_MB,CLOUDCOVER_AVG_PERCENT,HEATINDEX_MAX_C,DEWPOINT_MAX_C,WINDTEMP_MAX_C,date,WEATHER_CODE_EVENING,TOTAL_SNOW_MM,UV_INDEX,SUNHOUR,OPINION,flag,MONTH,DAY
0,2018-01-01,12,8,61,9,11,8,8.9,79,9.500,1018,41.750,12,8,7,2018-01-01,113.0,0.0,3.0,5.1,mauvais,False,1,1
1,2018-01-02,13,6,26,8,12,13,0.6,96,9.000,1020,87.875,13,12,6,2018-01-02,122.0,0.0,3.0,3.3,mauvais,False,1,2
2,2018-01-03,15,10,40,11,12,10,5.5,82,8.500,1017,91.500,15,13,7,2018-01-03,122.0,0.0,3.0,3.3,mauvais,False,1,3
3,2018-01-04,14,11,45,14,14,11,0.0,89,10.000,1011,90.125,14,12,10,2018-01-04,116.0,0.0,3.0,3.3,mauvais,False,1,4
4,2018-01-05,12,7,21,10,11,8,1.5,85,9.875,1005,62.375,12,10,7,2018-01-05,116.0,0.0,3.0,6.9,mauvais,False,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2018-12-27,7,2,6,2,6,5,0.0,85,10.000,1027,30.750,8,6,3,2018-12-27,119.0,0.0,1.0,8.7,très mauvais,False,12,27
361,2018-12-28,7,2,8,2,7,3,0.0,89,8.000,1035,18.750,8,4,4,2018-12-28,113.0,0.0,1.0,8.7,très mauvais,False,12,28
362,2018-12-29,7,1,6,1,6,4,0.0,94,7.000,1038,33.000,8,5,1,2018-12-29,116.0,0.0,1.0,8.7,très mauvais,False,12,29
363,2018-12-30,9,4,6,5,9,8,0.1,95,6.000,1038,70.375,10,9,7,2018-12-30,143.0,0.0,1.0,3.3,très mauvais,False,12,30


## Execution with a loop
You will execute your function in a 9 rounds loop, once per year, and agregate this in a new big DataFrame with 24 columns and lots of rows.

In [ ]:
# Each year between 2010 and 2018 (included) has 2 sources : weather_main_YYYY.csv and weather_opinion_YYYY.csv

# Initiate an empty DataFrame
dftotal = pd.DataFrame()

for var_year in range(2010, 2019):

    # Collect both DataFrames each year
    year = str(var_year)
    link_main = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather_main_" + year + ".csv"
    link_opinion = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather_opinion_" + year + ".csv"
    df_main = pd.read_csv(link_main)
    df_opinion = pd.read_csv(link_opinion)

    # Apply your merge, cleaning and fillna function on the current year
    df_year = merge_and_fill(df_main, df_opinion)

    # Concat with the global DF
    dftotal = pd.concat([dftotal , df_year])

dftotal.shape

(3287, 24)

## Add 2019 & 2020

In [ ]:
link2019 = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather2019.csv"
link2020 = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/weather2020.csv"

df_2019 = pd.read_csv(link2019)
df_2020 = pd.read_csv(link2020)

dftotal = pd.concat([dftotal , df_2019])
dftotal = pd.concat([dftotal , df_2020])
dftotal.shape

(4018, 26)

# Draw a scatterplot
With Date on X-Axis and Max Temperature on Y-axis

The result must have 11 seasonality for the 11 summers/winters alternance.




In [ ]:
fig = px.scatter(data_frame = dftotal, x= "DATE", y="MAX_TEMPERATURE_C", 
                 color = 'OPINION', title = "Températures et opinions", width=1300, height=700)
fig.update_layout(title_x = 0.5)
fig.show()


Find appropriate dataviz to answer to this question : 

Which year had the best weather? Are years very different or not ?


In [ ]:
#I don't really find any appropriate dataviz without modifying the datafirst especially as "OPINION" is the best way to find the year with the best weather.
#I also want to add a YEAR column

def find_year(date):
  transformed_date = datetime.strptime(date, '%Y-%m-%d').year
  return transformed_date

dftotal['YEAR'] = dftotal['DATE'].apply(find_year)

In [ ]:
# I give an arbitrary score to the opinion
dftotal2=dftotal #this is only for testing purpose later in the notebook

dftotal["OPINION"].replace({"très mauvais": 1, "very bad": 1, "mauvais" : 2, "bad": 2, "not good not bad": 3, 
                            "moyen" : 3, "bon": 4, "good": 4, "very good" : 5, "très bon" : 5, "unknown" : "inconnu"}, inplace=True)

In [ ]:
dftotal3 = dftotal2[["YEAR", "OPINION"]]
dftotal4 = dftotal3.value_counts().reset_index(name='counts')
dftotal4["score"] = dftotal4['OPINION'] * dftotal4['counts']

def remake_opinion_name(opinion_score):
  if opinion_score == 1:
    return "Très mauvais"
  if opinion_score == 2:
    return "Mauvais"
  if opinion_score == 3:
    return "Moyen"
  if opinion_score == 4:
    return "Bon"
  if opinion_score == 5:
    return "Très bon"

dftotal4['opinion_name'] = dftotal4['OPINION'].apply(remake_opinion_name)

dftotal6 = dftotal4.sort_values(by=['YEAR', 'OPINION'])

fig = px.bar(data_frame = dftotal6, x= "YEAR", y="score", title = "Score des opinions par années", color="OPINION", width=1300, height=700, template='plotly_dark', text="opinion_name",
             color_discrete_map={5: "green",4: "lightgreen",3: "yellow",2: "orange", 1: "red", "inconnu": "grey"})

fig.update_xaxes(tickmode='linear', title="")
fig.update_yaxes(title="")
fig.update_layout(title_x = 0.5, showlegend = False)
fig.show()

L'année ayant eu le meilleur score d'opinion est l'année 2014. On peut voir que les différences ne sont globalement pas trop marquées.

In [ ]:
dftotal7 = dftotal6[dftotal6['OPINION'] != 'inconnu'] #retrait des opinions inconnus
dftotal8 = dftotal7[["YEAR", "score"]]
dftotal9 = dftotal8.groupby(['YEAR']).sum().sort_values(by=['score'], ascending=False)
dftotal9.reset_index(inplace=True)
dftotal9

,YEAR,score
0,2014,987
1,2011,969
2,2020,929
3,2015,915
4,2017,915
5,2018,913
6,2019,906
7,2016,884
8,2012,873
9,2010,860


In [ ]:
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
###older version
fig = px.bar(data_frame = dftotal, x= "YEAR", y="OPINION", title = "Score des opinions par années", color="OPINION", width=1300, height=700, template='plotly_dark',
             color_discrete_map={5: "green",4: "lightgreen",3: "yellow",2: "orange", 1: "red", "inconnu": "grey"})

fig.update_layout(title_x = 0.5)
fig.show()